In [16]:
import pandas as pd
import json
import nltk as nlt 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlt.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nico\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [23]:
ruta="../Dataset_Original/australian_user_reviews.json"

In [62]:
with open(ruta, 'r', encoding='utf-8') as w:
    data = w.readlines() #la funcion eval se utiliza para evaluar expresiones de cadena que no son seguras, esto se hace así debido a que no es un json puro el json que estamos manejando  
    data = [eval(line.strip()) for line in data]
df=pd.DataFrame(data)

In [63]:
#exploramos el dataframe
df

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [64]:
#ahora desanidamos la coluna reviews y hacemos otro dataframe que contenga el user_id de la persona que hizo cada review

#para esto reutilizamos la variable data que fue en la que al principio leimos el json

reviews=pd.json_normalize(data, record_path="reviews" , meta="user_id")
reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley


In [65]:
#borramos las columnas que no vamos a utilizar 
reviews=reviews.drop(["funny", "last_edited","helpful"], axis=1)
reviews

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
59300,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244
59301,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244
59302,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,Posted July 20.,730,True,:D,LydiaMorley


In [66]:
# inicializamos el analisis de sentimientos
sia = SentimentIntensityAnalyzer()

# con esta funcion convertimos cada comentario en un valor de 0 a 2 de acuerdo a la escala pedida (0 como valoración negativa, 1 como valoración neutral y 2 como valoración positiva)
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # en el caso de que el comentario sea null o NaN retornamos 1, es decir valoración neutral
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # buena valoración
        elif compound_score <= -0.05:
            return 0  # mala valoración
        else:
            return 1
    else:
        return 1  # valoración neutral
    
    
    
# ahora modificamos la columna "posted" y luego agregamos la columna 
# sentiment_score, la cual se construye mediante la columna review

# para hacer esto convertimos la columna 'review' a tipo de dato str
    
reviews['review'] = reviews['review'].astype(str)

# aplicamos la función get_sentiment_score a la columna 'review'
reviews['sentiment_score'] = reviews['review'].apply(get_sentiment_score)


In [67]:
#convertimos la columna posted a string
reviews['posted'] = reviews['posted'].str.replace('Posted', '')

In [68]:
# reemplazamos los valores no válidos por NaN
reviews["posted"] = pd.to_datetime(reviews["posted"], errors='coerce')
# filtramos el DataFrame para obtener los registros válidos
df_filtered = reviews.dropna(subset=["posted"])
# convertimos la columna "posted" a tipo datetime
df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])
# rellenamos los valores faltantes en las columna'posted' con la moda
columns_fill_mode = ['posted']
for col in columns_fill_mode:
    reviews[col].fillna(reviews[col].mode()[0], inplace=True)
# modelamos y eliminamos de la columna 'posted'
reviews['posted_year'] = reviews['posted'].dt.year.astype('Int64')

#finalmente borramos las columnas "posted" y "review" ya que no las utilizaremos más
reviews.drop('posted', axis=1, inplace=True)
reviews.drop('review', axis=1, inplace=True)

C:\Users\nico\AppData\Local\Temp\ipykernel_8568\2390520892.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["posted"] = pd.to_datetime(df_filtered["posted"])


In [70]:
#finalmente guardamos el dataframe reviews en un csv
reviews.to_csv("reviews.csv")